In [1]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


SEARCH_WORD = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "アスカ狭山店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    }

# 検索キーワードよりホール名取得
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{SEARCH_WORD}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"], cherry=True).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
df["area"] = df["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
print(f'以下の日付のデータが含まれています')
print(df.date.unique()[0])
model_list = df["model_name"].unique()
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}', end=", ")
    
df.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 60952 x 17
以下の日付のデータが含まれています
2025-05-08 00:00:00
以下のモデルが含まれています
0: ゴーゴージャグラー3, 1: マイジャグラーV, 2: ファンキージャグラー2, 3: アイムジャグラーEX-TP, 4: ミスタージャグラー, 5: ウルトラミラクルジャグラー, 6: ハッピージャグラーVIII, 7: ジャグラーガールズ, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
0,EXA FIRST,2025-05-08,ゴーゴージャグラー3,1001,701,3,0,-238,233.7,0.0,6.76,233.7,2025-05,8,3,1,a: 1001-
1,EXA FIRST,2025-05-08,ゴーゴージャグラー3,1002,3544,9,15,-876,393.8,236.3,6.21,147.7,2025-05,8,3,2,a: 1001-
2,EXA FIRST,2025-05-08,ゴーゴージャグラー3,1003,447,0,0,-603,0.0,0.0,6.68,0.0,2025-05,8,3,3,a: 1001-
3,EXA FIRST,2025-05-08,ゴーゴージャグラー3,1004,718,2,1,-326,359.0,718.0,6.18,239.3,2025-05,8,3,4,a: 1001-
4,EXA FIRST,2025-05-08,ゴーゴージャグラー3,1005,7943,24,32,-1018,331.0,248.2,6.10,141.8,2025-05,8,3,5,a: 1001-


## 過去7日間の差枚と翌日の結果を比較

In [3]:
def extract_and_merge_model_data(df, model_name, start_date, end_date):
    # 対象期間・モデルのデータを抽出
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered["date"].dt.date <= start_date) & (df_filtered["date"].dt.date >= end_date)]
    df_filtered = df_filtered[(df_filtered["model_name"] == model_name)]
    
    # 各種ピボットテーブル
    medals = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="medals", aggfunc="sum")
    game = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="game", aggfunc="sum",)
    rb_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="RB_rate", aggfunc="sum")
    total_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="Total_rate", aggfunc="sum")
    grape_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="Grape_rate", aggfunc="sum")
    
    # 日付列を反転・スライス
    medals = medals.iloc[:, 7:].iloc[:, ::-1]
    game = game.iloc[:, 7:].iloc[:, ::-1]
    rb_rate = rb_rate.iloc[:, 7:].iloc[:, ::-1]
    total_rate = total_rate.iloc[:, 7:].iloc[:, ::-1]
    grape_rate = grape_rate.iloc[:, 7:].iloc[:, ::-1]

    # 7日間累積とランク
    # rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
    # rolling_7d_sum.columns = [f"{col.strftime('%y-%m-%d')}_7d_sum" for col in rolling_7d_sum.columns]
    # rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]
    # rolling_7d_rank = rolling_7d_sum.rank(method="min", ascending=True).fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    # rolling_7d_rank.columns = [c.replace("sum", "rank") for c in rolling_7d_rank.columns]
    # rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]
    # rolling_7d_rank = rolling_7d_rank.iloc[:, ::-1]

    # MultiIndex化（ラベル付け）
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])
    grape_rate.columns = pd.MultiIndex.from_product([["GRAPE_RATE"], grape_rate.columns])
    # rolling_7d_sum.columns = pd.MultiIndex.from_product([["7D_sum"], rolling_7d_sum.columns])
    # rolling_7d_rank.columns = pd.MultiIndex.from_product([["RANK"], rolling_7d_rank.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols  = [
        col for pair in zip(
            # rolling_7d_rank.columns,
            # rolling_7d_sum.columns,
            medals.columns,
            game.columns,
            rb_rate.columns,
            total_rate.columns,
            grape_rate.columns
            ) for col in pair
        ]
    merged = pd.concat([
        # rolling_7d_rank, rolling_7d_sum, 
        medals, game, rb_rate, total_rate, grape_rate], 
                       axis=1
        )[interleaved_cols]
    merged = merged[~merged.iloc[:, 2].isna()]

    # エリアごとに空行挿入して整形
    merged_by_area = pd.DataFrame()
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        if not area_merged.empty:
            empty_index = pd.MultiIndex.from_tuples([("", " ", " ")], names=merged.index.names)
            empty_row = pd.DataFrame([[""] * area_merged.shape[1]], index=empty_index, columns=area_merged.columns)
            merged_by_area = pd.concat([merged_by_area, area_merged, empty_row])
    # インデックス削除
    merged_by_area = merged_by_area.droplevel("area")
    
    return merged_by_area


today = datetime.date.today()
start_date = today - relativedelta(days=1)
end_date = start_date - relativedelta(days=30)

merged_by_model = pd.DataFrame()
model_list = ['マイジャグラーV', 'アイムジャグラーEX-TP', 'ゴーゴージャグラー3', 'ファンキージャグラー2', 'ミスタージャグラー' , "ハッピージャグラーVIII",]
for model in model_list:
    merged_by_area = extract_and_merge_model_data(df, model, start_date, end_date)
    # モデル間の区切り用空行追加して結合
    empty_index = pd.MultiIndex.from_tuples([(" ", " ")], names=merged_by_area.index.names)
    empty_row = pd.DataFrame([[""] * merged_by_area.shape[1]], index=empty_index, columns=merged_by_area.columns)
    merged_by_model = pd.concat([merged_by_model, merged_by_area, empty_row], axis=0)
# 出力
merged_by_model.to_csv(f"merged_by_model_{start_date}_{end_date}.csv", encoding="utf_8_sig")
display(merged_by_model)

MEDALS       GAME    RB_RATE TOTAL_RATE GRAPE_RATE  \
date                  2025-05-08 2025-05-08 2025-05-08 2025-05-08 2025-05-08   
model_name    unit_no                                                          
マイジャグラーV      1021        -656.0      860.0      430.0      286.7       6.23   
              1022        -374.0     3836.0      348.7      159.8       5.86   
              1023        -515.0     3026.0      302.6      159.3       5.93   
              1024         326.0     3732.0      622.0      162.3       5.75   
              1025        -432.0     1476.0      295.2      147.6       7.14   
...                          ...        ...        ...        ...        ...   
ハッピージャグラーVIII 1141        -250.0     2067.0      689.0      187.9       5.88   
              1142        -268.0      698.0      349.0      174.5       6.64   
              1143        1394.0     4355.0      256.2      117.7       5.73   
                                                                               
                                                                               

                          MEDALS       GAME    RB_RATE TOTAL_RATE GRAPE_RATE  \
date                  2025-05-07 2025-05-07 2025-05-07 2025-05-07 2025-05-07   
model_name    unit_no                                                          
マイジャグラーV      1021        1738.0     7324.0      318.4      128.5       5.82   
              1022        -709.0     1873.0      936.5      234.1       6.15   
              1023        -438.0     1696.0      565.3      212.0       5.66   
              1024         924.0     6381.0      490.8      145.0       5.99   
              1025         594.0     5867.0      391.1      143.1       5.95   
...                          ...        ...        ...        ...        ...   
ハッピージャグラーVIII 1141          26.0     1878.0      469.5      170.7       5.46   
              1142         250.0     3498.0      269.1      134.5       5.82   
              1143        -362.0     1941.0      647.0      176.5       6.52   
                                                                               
                                                                               

                       ...     MEDALS       GAME    RB_RATE TOTAL_RATE  \
date                   ... 2025-04-16 2025-04-16 2025-04-16 2025-04-16   
model_name    unit_no  ...                                               
マイジャグラーV      1021     ...      -91.0     3044.0      434.9      169.1   
              1022     ...      359.0     5413.0      360.9      142.4   
              1023     ...     -750.0     1230.0     1230.0      410.0   
              1024     ...     -950.0     4199.0      466.6      200.0   
              1025     ...     1571.0     6256.0      312.8      130.3   
...                    ...        ...        ...        ...        ...   
ハッピージャグラーVIII 1141     ...        NaN        NaN        NaN        NaN   
              1142     ...        NaN        NaN        NaN        NaN   
              1143     ...        NaN        NaN        NaN        NaN   
                       ...        NaN        NaN        NaN        NaN   
                       ...        NaN        NaN        NaN        NaN   

                      GRAPE_RATE     MEDALS       GAME    RB_RATE TOTAL_RATE  \
date                  2025-04-16 2025-04-15 2025-04-15 2025-04-15 2025-04-15   
model_name    unit_no                                                          
マイジャグラーV      1021          5.58     1791.0     9038.0      251.1      127.3   
              1022           6.0      124.0     5877.0      326.5      146.9   
              1023          5.44    -1465.0     6460.0      239.3      150.2   
              1024          5.58     -468.0     7420.0      322.6      154.6   
              1025          5.62    -1606.0     2240.0      448.0      280.0   
...                          ...        ...        ...        ...        ...   
ハッピージャグラーVIII 1141         

## 2024年のGWの分析

In [406]:
# model_name = model_list[1]
today = datetime.date.today()
target_date = today - datetime.timedelta(days=1)
start_date = datetime.date(2024, 5, 5)
end_date = datetime.date(2024, 4, 27)

merged_by_model = pd.DataFrame()
model_list = ['マイジャグラーV', 'アイムジャグラーEX-TP', 'ゴーゴージャグラー3', 'ファンキージャグラー2', 'ミスタージャグラー' ]
for model in model_list:
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered["date"].dt.date <= start_date) & (df_filtered["date"].dt.date >= end_date)]
    df_filtered = df_filtered[(df_filtered["model_name"] == model)]